In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.vision.all import *
import sklearn.metrics as skm
from tqdm.notebook import tqdm
import sklearn.feature_extraction.text
from transformers import AutoModelForMaskedLM, AutoConfig, BertTokenizer, LineByLineTextDataset

from shopee_utils import *
from train_utils import *
import codecs
from torch.utils.data.dataset import Dataset

In [3]:
BERT_CONFIG_PATH = './indobert-large-p2'
PATH = Path('/home/slex/data/shopee')

In [4]:
train_df = pd.read_csv(PATH/'train.csv')

In [5]:
model = AutoModelForMaskedLM.from_pretrained(BERT_CONFIG_PATH)

Some weights of BertForMaskedLM were not initialized from the model checkpoint at ./indobert-large-p2 and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tokenizer = BertTokenizer.from_pretrained(BERT_CONFIG_PATH)

In [7]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, lines, block_size):
        batch_encoding = tokenizer(lines, add_special_tokens=True, truncation=True, max_length=block_size)
        self.examples = batch_encoding["input_ids"]
        self.examples = [{"input_ids": torch.tensor(e, dtype=torch.long)} for e in self.examples]
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [8]:
dataset=TextDataset(tokenizer, train_df.title.to_list(), 128)

In [9]:
tokenizer.decode(dataset[0]['input_ids'])

'[CLS] paper bag victoria secret [SEP]'

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./LanguageModel",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_gpu_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


In [12]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
  2%|▏         | 501/21420 [01:39<1:04:03,  5.44it/s]

{'loss': 2.5538, 'learning_rate': 4.88328664799253e-05, 'epoch': 0.47}


  5%|▍         | 1001/21420 [03:15<55:51,  6.09it/s] 

{'loss': 1.8734, 'learning_rate': 4.7665732959850615e-05, 'epoch': 0.93}


  7%|▋         | 1500/21420 [04:52<1:10:44,  4.69it/s]

...

 98%|█████████▊| 21001/21420 [1:10:01<01:14,  5.59it/s]

{'loss': 0.669, 'learning_rate': 9.80392156862745e-07, 'epoch': 19.61}


100%|██████████| 21420/21420 [1:11:24<00:00,  5.00it/s]

{'train_runtime': 4284.635, 'train_samples_per_second': 4.999, 'epoch': 20.0}


TrainOutput(global_step=21420, training_loss=0.9839220048777975, metrics={'train_runtime': 4284.635, 'train_samples_per_second': 4.999, 'epoch': 20.0})

In [14]:
trainer.save_model('./finetuned_indobert-large-p2')